All the imports 

In [112]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, datasets
import os
from PIL import Image

In [113]:
print(torch.__version__)

2.3.1+cpu


In [114]:
class HomerBartDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label, sub_dir in enumerate(os.listdir(root_dir)):
            sub_dir_path = os.path.join(root_dir, sub_dir)
            for img_name in os.listdir(sub_dir_path):
                self.image_paths.append(os.path.join(sub_dir_path, img_name))
                self.labels.append(label)
                
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [115]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])


In [116]:
dataset = HomerBartDataset(root_dir='homer_bart', transform=transform)

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [117]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(64 * 64 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

model = SimpleNN()


In [118]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00005)


In [119]:
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


Epoch 1, Loss: 0.6723271682858467
Epoch 2, Loss: 0.6326306983828545
Epoch 3, Loss: 0.6095225661993027
Epoch 4, Loss: 0.5822339504957199
Epoch 5, Loss: 0.588665209710598
Epoch 6, Loss: 0.5684722438454628
Epoch 7, Loss: 0.5648433417081833
Epoch 8, Loss: 0.5521713979542255
Epoch 9, Loss: 0.540608361363411
Epoch 10, Loss: 0.5518081001937389
Epoch 11, Loss: 0.5291669443249702
Epoch 12, Loss: 0.5186579376459122
Epoch 13, Loss: 0.5163976848125458
Epoch 14, Loss: 0.5348396189510822
Epoch 15, Loss: 0.5069563388824463
Epoch 16, Loss: 0.4986739084124565
Epoch 17, Loss: 0.504387617111206
Epoch 18, Loss: 0.4976184666156769
Epoch 19, Loss: 0.476128164678812
Epoch 20, Loss: 0.5032254606485367
Epoch 21, Loss: 0.47716081887483597
Epoch 22, Loss: 0.4847048707306385
Epoch 23, Loss: 0.4728654697537422
Epoch 24, Loss: 0.46974874660372734
Epoch 25, Loss: 0.47473715618252754
Epoch 26, Loss: 0.4643835462629795
Epoch 27, Loss: 0.4581010974943638
Epoch 28, Loss: 0.45890340954065323
Epoch 29, Loss: 0.44505126774

In [120]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total}%')


Test Accuracy: 92.5925925925926%
